In [1]:
import numpy as np
import scipy
import sklearn
import scipy.stats as stats
import pandas as pd
import fc_subtypes_functions as func


from sklearn import datasets
from statsmodels.stats import multitest
from copy import deepcopy




In [2]:
# Load Weights and intitialze variables

networks = [ 'CER','LIM','MOT','VIS','DMN','FP','VATTSAL'] 
labels = ["1","2","3","4","5","6","7"]

%store -r w_
weights = w_
num_subtypes = 7


%store -r selection_demog
selected_subjects = selection_demog.index

networks_weights_dict = dict()
contingency_table_dict = dict()

demo = pd.read_csv("adni_cimaq_info_niak.csv").rename(columns = {'Unnamed: 0': "subjectID"})
demo = demo[list(demo.columns[0:11])]


In [3]:

for net in labels:
    network = int(net)-1
    label_network = networks[int(net)-1]
    
    #Create a dataframe for each network
    pd_weights = pd.DataFrame(columns = ["subjectID"])
    
    pd_weights["subjectID"] = selected_subjects
    
    for subtype_ in range(7):
        subtype_curr = "Subtype"+str(subtype_+1)+"_"+label_network
        print(subtype_curr,)
        weight = weights[:,network*num_subtypes+subtype_]
        pd_weights[subtype_curr] = weight
        
    
    print(" ")
    #create a dataframe for each network 
    networks_weights_dict.update({label_network:pd_weights})
    networks_weights_dict[label_network] = func.max_subtype(networks_weights_dict[label_network])
    
    #merge with demographic information 
    networks_weights_dict[label_network] = networks_weights_dict[label_network].merge(demo, how="left" ,on="subjectID")
    
    #create a contingency table for each network 
    contingency_table = func.create_ct(networks_weights_dict[label_network],num_subtypes)
    contingency_table_dict.update({label_network:contingency_table})

Subtype1_CER
Subtype2_CER
Subtype3_CER
Subtype4_CER
Subtype5_CER
Subtype6_CER
Subtype7_CER
 
Subtype1_LIM
Subtype2_LIM
Subtype3_LIM
Subtype4_LIM
Subtype5_LIM
Subtype6_LIM
Subtype7_LIM
 
Subtype1_MOT
Subtype2_MOT
Subtype3_MOT
Subtype4_MOT
Subtype5_MOT
Subtype6_MOT
Subtype7_MOT
 
Subtype1_VIS
Subtype2_VIS
Subtype3_VIS
Subtype4_VIS
Subtype5_VIS
Subtype6_VIS
Subtype7_VIS
 
Subtype1_DMN
Subtype2_DMN
Subtype3_DMN
Subtype4_DMN
Subtype5_DMN
Subtype6_DMN
Subtype7_DMN
 
Subtype1_FP
Subtype2_FP
Subtype3_FP
Subtype4_FP
Subtype5_FP
Subtype6_FP
Subtype7_FP
 
Subtype1_VATTSAL
Subtype2_VATTSAL
Subtype3_VATTSAL
Subtype4_VATTSAL
Subtype5_VATTSAL
Subtype6_VATTSAL
Subtype7_VATTSAL
 


In [4]:
network_interest = "LIM" 

In [5]:
networks_weights_dict[network_interest].head()

,subjectID,Subtype1_LIM,Subtype2_LIM,Subtype3_LIM,Subtype4_LIM,Subtype5_LIM,Subtype6_LIM,Subtype7_LIM,max_weight,max_subtype,age,Female,CN,SCI,MCI,AD,QC,FD_scrubbed,adni2,cimaq
0,s0107,0.197057,0.125242,-0.249710,-0.058888,0.081640,0.217317,-0.075809,-0.249710,Subtype3_LIM,67.0,1,0,0,1,0,1,0.153941,1,0
1,s0186,-0.110283,-0.071080,0.055686,-0.059796,0.110551,-0.017179,-0.012178,0.110551,Subtype5_LIM,87.0,1,1,0,0,0,1,0.173473,1,0
2,s0295,0.305625,0.175440,-0.341655,-0.320116,0.192116,0.394088,-0.098200,0.394088,Subtype6_LIM,90.0,0,1,0,0,0,1,0.246640,1,0
3,s0685,-0.067502,-0.024334,0.084277,-0.147075,0.146719,-0.068824,-0.026451,-0.147075,Subtype4_LIM,94.0,1,1,0,0,0,1,0.209432,1,0
4,s0729,0.086681,0.104812,-0.193715,-0.145134,0.293672,0.070670,-0.010623,0.293672,Subtype5_LIM,70.0,1,0,0,0,1,1,0.191809,1,0


In [6]:
contingency_table_dict[network_interest]

,class,Subtype1_LIM,Subtype2_LIM,Subtype3_LIM,Subtype4_LIM,Subtype5_LIM,Subtype6_LIM,Subtype7_LIM
0,CN,5,7,10,7,16,12,1
1,ADMCI,10,13,32,25,12,16,7


# Discrete Associations


In [7]:
discrete_associations = pd.DataFrame(columns = ["network","pchi","qfdr","chiV"])
pchi_arr = []
qfdr_arr = []
chiV_arr = []

for net in networks:


    ct_table = deepcopy(contingency_table_dict[net])
    del ct_table["class"]
    pchi = stats.chi2_contingency(ct_table.values, correction=False, lambda_=None)
    pchi_arr.append(pchi[1])
    qfdr_arr.append(multitest.multipletests(pchi[1],0.05,'fdr_bh')[1][0])
    chiV_arr.append(func.cramers_corrected_stat(ct_table.values,pchi[0]))
    
    
discrete_associations["network"] =  networks  
discrete_associations["pchi"] =  pchi_arr  
discrete_associations["qfdr"] =  qfdr_arr  
discrete_associations["chiV"] =  chiV_arr  


discrete_associations

,network,pchi,qfdr,chiV
0,CER,0.954757,0.954757,0.000000
1,LIM,0.036603,0.036603,0.207448
2,MOT,0.415342,0.415342,0.014386
3,VIS,0.868157,0.868157,0.000000
4,DMN,0.558996,0.558996,0.000000
5,FP,0.628973,0.628973,0.000000
6,VATTSAL,0.789605,0.789605,0.000000



# Continuous Associations


In [17]:

for net in networks:
    for subtype_x in networks_weights_dict[net].columns[1:8]:
        pd_network = networks_weights_dict[net]
        X = pd_network[["CN"]].values
        y = pd_network[[subtype_x]].values
        scipy.io.savemat('./glm_mat/subtypes7/'+subtype_x+'.mat', dict(x=X, y=y))  
        


In [19]:
glm_output = pd.read_csv("glm_mat/subtypes7/glm_output_7.csv")
glm_output

,,ttest,pce
0,Subtype1_CER,0.474843,0.635353
1,Subtype2_CER,0.950176,0.343029
2,Subtype3_CER,-0.319591,0.749571
3,Subtype4_CER,-0.843605,0.399775
4,Subtype5_CER,-0.253731,0.799932
5,Subtype6_CER,-0.328037,0.743185
6,Subtype7_CER,-0.359997,0.719183
7,Subtype1_LIM,0.064087,0.948957
8,Subtype2_LIM,0.851135,0.395587
9,Subtype3_LIM,-0.308678,0.757849
